<a href="https://colab.research.google.com/github/shirleyzz/cap5610_machineleanrning/blob/master/HW2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#using logistic regression with cross entropy
from keras.datasets import mnist
import numpy as np
from keras import backend as K
import keras
from keras.utils import to_categorical

In [0]:
(x_train_origin, y_train_origin),(x_test_origin, y_test_origin) = mnist.load_data()

In [0]:
#classifier of flag, equals target number
def convert_labels(df,flag=0):
  for item in range(df.shape[0]):
    if df[item] == flag:
      df[item] = 1
    else:
      df[item] = 0
  return df

In [0]:
from imblearn.over_sampling import RandomOverSampler
def preprocessing(flag=0,oversampling=False):
  x_train = x_train_origin.reshape((60000, 28 * 28))
  x_test = x_test_origin.reshape((10000, 28 * 28))
  #set the flag to be the one classifier you choose
  train_labels = convert_labels(y_train_origin,flag)
  test_labels = convert_labels(y_test_origin,flag)
  #set oversampling
  if(oversampling):
    ros = RandomOverSampler(random_state=3)
    x_train, train_labels = ros.fit_resample(x_train, train_labels)
 # print(x_train.shape)
  train_labels = train_labels.reshape(1,train_labels.shape[0])
  test_labels = test_labels.reshape(1,test_labels.shape[0])
  x_train = x_train.reshape(x_train.shape[0],-1).T
  x_test = x_test.reshape(x_test.shape[0],-1).T
  x_train = x_train.astype('float32') / 255.0
  x_test = x_test.astype('float32') / 255.0
  #print(x_train.shape)
  #check whether labels are good
  #print(len(train_labels[train_labels==1]))
  return x_train,train_labels,x_test,test_labels

Each of the ten classifiers has an input layer consisting of 28 x 28 input neurons and an output layer consisting of a single output neuron.





In [0]:
def sigmoid(x):
  z = 1.0/(1.0+np.exp(-x))
  return z

In [0]:
def w_b_initialize(dim):
    w = np.zeros((dim,1))
    b = 0  
    return w, b

In [0]:
def forward_propagate(w, b, X, Y):
    m = X.shape[1]
    #print("m shape = ", str(m))
    A = sigmoid(np.dot(w.T, X) + b)
    loss = -1./m * np.sum(Y * np.log(A) + (1 - Y) * np.log(1-A))
    dw = (1/m)*np.dot(X,(A-Y).T)
    db = 1/m*np.sum(A-Y)
    
    loss = np.squeeze(loss)
    grads = {"dw": dw,
             "db": db}
    return grads, loss

In [0]:
def optimize(w, b, X, Y, num_iters, batch_size, learning_rate, print_cost):
   
    costs = []
    m = X.shape[1]
    for i in range(num_iters):
        #cost_batch is used to collect the cost during one iteration over different batches
        cost_batch = []
        #stochastic gradient descent
        shuffled_indices = np.random.permutation(m)
        X_shuffled = X[:,shuffled_indices]
        y_shuffled = Y[:,shuffled_indices]
        #apply mini-batch
        for j in range(0, m, batch_size):
            x_batch = X_shuffled[:, j:j+batch_size]
            y_batch = y_shuffled[:, j:j+batch_size]
            grads, cost = forward_propagate(w, b, x_batch, y_batch)
            dw = grads["dw"]
            db = grads["db"]
            w = w - learning_rate *  dw    
            b = b - learning_rate *  db
            cost_batch.append(cost)
            # the cost of one iteration is the average number over batches
            cost = np.mean(cost_batch)
        costs.append(cost)            
        if print_cost:
            print ("Cost after iteration %i: %f" % (i+1, cost))
    
    params = {"w": w,
              "b": b}
    
    grads = {"dw": dw,
             "db": db}
    
    return params, grads, costs

In [0]:
#  predict
def predict(w, b, X):
    m = X.shape[1]
    Y_pred = np.zeros((1, m))
    w = w.reshape(X.shape[0], 1)
    
    
    A = sigmoid(np.dot(w.T, X) + b)
       
    for i in range(A.shape[0]):
        if A[0][i] <= 0.5:
            Y_pred[0, i] = 0
        else:
            Y_pred[0, i] = 1
    return Y_pred

In [0]:
def model(X_train, Y_train, X_test, Y_test, num_iters=2000, batch_size=20000, learning_rate=0.5, print_cost=False):

    
    w, b = w_b_initialize(X_train.shape[0])                          

    parameters, grads, costs = optimize(w, b, X_train, Y_train, num_iters,batch_size, learning_rate, print_cost=False)     
   

    w = parameters["w"]
    b = parameters["b"]
    
    # predict test/train set examples 
    Y_pred_test = predict(w, b, X_test)
    Y_pred_train = predict(w, b, X_train)

    # print train/test Errors
    print("")
    print("train accuracy: {} %".format(100 - np.mean(np.abs(Y_pred_train - Y_train)) * 100))
    print("test accuracy: {} %".format(100 - np.mean(np.abs(Y_pred_test - Y_test)) * 100))
    repr('train accuracy')
  
    d = {"costs": costs,
         "Y_pred_test": Y_pred_test, 
         "Y_pred_train" : Y_pred_train, 
         "w" : w, 
         "b" : b,
         "learning_rate" : learning_rate,
         "num_iters": num_iters}
    
    return d

In [54]:
#choose number=0, implement model without over-sampling
x_train,train_labels,x_test,test_labels = preprocessing()
d = model(x_train, train_labels, x_test, test_labels, num_iters=10, batch_size=10000, learning_rate=0.01, print_cost=True)


train accuracy: 90.12833333333333 %
test accuracy: 90.2 %


In [55]:
#choose number=1, implement model with over-sampling
x_train,train_labels,x_test,test_labels = preprocessing(flag=1,oversampling=True)
d = model(x_train, train_labels, x_test, test_labels, num_iters=10,batch_size=20000, learning_rate=0.01, print_cost=True)


train accuracy: 50.0 %
test accuracy: 90.2 %
